In [4]:
# --- 1. 라이브러리 설치 및 임포트 ---

import pandas as pd
import numpy as np
import os
from IPython.display import display

In [ ]:
# --- 2. 경로 설정 및 데이터 로드 ---

# 학습 데이터 파일 경로
TRAIN_DATA_PATH = '../../data/raw/train.csv'
TEST_DATA_PATH = '../../data/raw/test.csv'
SUBWAY_DATA_PATH = '../../data/raw/subway_feature.csv'
BUS_DATA_PATH = '../../data/raw/bus_feature.csv'

# 출력 파일 경로 설정
FEATURE_DIR = '../../data/processed/transportation-features/'
TRAIN_FEATURE_FILENAME = 'train_transportation_features.csv'
TRAIN_FEATURE_PATH = os.path.join(FEATURE_DIR, TRAIN_FEATURE_FILENAME)
TEST_FEATURE_FILENAME = 'test_transportation_features.csv'
TEST_FEATURE_PATH = os.path.join(FEATURE_DIR, TEST_FEATURE_FILENAME)

# 데이터 로드
print("데이터를 로드합니다...")

try:
    # pandas에서 dtype 관련 경고 제거를 위해 low_memory=False 설정
    train_df = pd.read_csv(TRAIN_DATA_PATH, low_memory=False)
    test_df = pd.read_csv(TEST_DATA_PATH, low_memory=False)
    subway_df = pd.read_csv(SUBWAY_DATA_PATH)
    bus_df = pd.read_csv(BUS_DATA_PATH)
    
    print("모든 데이터 로드 완료.")
    # 데이터가 정상적으로 로드되었음을 표시
    data_loaded_successfully = True
    
except FileNotFoundError as e:
    print(f"오류: 파일 로드에 실패했습니다. 경로를 확인해주세요. \n>> {e}")
    data_loaded_successfully = False

데이터를 로드합니다...
모든 데이터 로드 완료.


In [20]:
# --- 3. Haversine 거리 계산 함수 정의 ---

def haversine_distance_vectorized(lon1, lat1, lon2, lat2):
    # 입력을 numpy array로 변환 (Series/ndarray 모두 안전 처리)
    lon1_arr = np.asarray(lon1)
    lat1_arr = np.asarray(lat1)
    lon2_arr = np.asarray(lon2)
    lat2_arr = np.asarray(lat2)

    # 라디안 변환 및 reshape
    lon1_rad = np.radians(lon1_arr).reshape(-1, 1)
    lat1_rad = np.radians(lat1_arr).reshape(-1, 1)
    lon2_rad = np.radians(lon2_arr).reshape(1, -1)
    lat2_rad = np.radians(lat2_arr).reshape(1, -1)

    # 하버사인 공식
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c  # 단위: km


In [23]:
# --- 4. 파생피처 생성 함수 정의 ---
def create_transportation_features(df, subway_df, bus_df):
    # 거리 행렬 계산
    dist_sub = haversine_distance_vectorized(
        df['좌표X'], df['좌표Y'],
        subway_df['경도'], subway_df['위도']
    )
    dist_bus = haversine_distance_vectorized(
        df['좌표X'], df['좌표Y'],
        bus_df['X좌표'], bus_df['Y좌표']
    )
    # 딕셔너리로 정리
    features = {
        'ID': df['아파트명'],
        '지하철최단거리':         dist_sub.min(axis=1),
        '반경_1km_지하철역_수':   (dist_sub < 1.0).sum(axis=1),
        '반경_500m_지하철역_수':  (dist_sub < 0.5).sum(axis=1),
        '반경_300m_지하철역_수':  (dist_sub < 0.3).sum(axis=1),
        '버스최단거리':           dist_bus.min(axis=1),
        '반경_1km_버스정류장_수':  (dist_bus < 1.0).sum(axis=1),
        '반경_500m_버스정류장_수': (dist_bus < 0.5).sum(axis=1),
        '반경_300m_버스정류장_수': (dist_bus < 0.3).sum(axis=1),
    }
    return pd.DataFrame(features)

In [24]:
# --- 5. train/test 파생피처 생성 및 미리보기 ---
# train_features_df = create_transportation_features(train_df, subway_df, bus_df)
# print("\ntrain 파생피처 상위 5개:")
# display(train_features_df.head())


test_features_df  = create_transportation_features(test_df, subway_df, bus_df)
print("\ntest 파생피처 상위 5개:")
display(test_features_df.head(10))


test 파생피처 상위 5개:


,ID,지하철최단거리,반경_1km_지하철역_수,반경_500m_지하철역_수,반경_300m_지하철역_수,버스최단거리,반경_1km_버스정류장_수,반경_500m_버스정류장_수,반경_300m_버스정류장_수
0,개포6차우성,1.129775,0,0,0,0.061783,58,13,7
1,개포더샵트리에,0.306673,5,1,0,0.125972,72,15,8
2,개포우성3차,0.413746,4,1,0,0.098282,77,19,9
3,개포우성3차,0.413746,4,1,0,0.098282,77,19,9
4,개포우성3차,0.413746,4,1,0,0.098282,77,19,9
5,개포주공5단지,0.221601,4,1,1,0.131192,59,22,11
6,개포주공6단지,0.250322,4,1,1,0.094096,62,20,11
7,개포주공6단지,0.250322,4,1,1,0.094096,62,20,11
8,개포주공6단지,0.250322,4,1,1,0.094096,62,20,11
9,래미안블레스티지,0.887864,1,0,0,0.050355,49,12,6


In [25]:
# --- 5. 결과 저장 ---

# 파생변수 데이터프레임이 정상적으로 생성되었는지 확인 후 저장
# if train_features_df is not None:
#     try:
#         # 파생변수 DataFrame을 CSV로 저장
#         train_features_df.to_csv(TRAIN_FEATURE_PATH, index=False)
#         print(f"🎉 train.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '{TRAIN_FEATURE_PATH}'")

#     except Exception as e:
#         print(f"\n오류: 파일 저장 중 문제가 발생했습니다. \n>> {e}")
# else:
#     print("저장할 파생변수 데이터가 없습니다. (셀 1)을 먼저 실행해주세요.")
    

if test_features_df is not None:
    try:
        # 파생변수 DataFrame을 CSV로 저장
        test_features_df.to_csv(TEST_FEATURE_PATH, index=False)
        print(f"🎉 test.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '{TEST_FEATURE_PATH}'")

    except Exception as e:
        print(f"\n오류: 파일 저장 중 문제가 발생했습니다. \n>> {e}")
else:
    print("저장할 파생변수 데이터가 없습니다. (셀 1)을 먼저 실행해주세요.")

🎉 test.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '../../data/processed/transportation-features/train_transportation_test_features.csv'
